# 1. Scrape

In [ ]:
# Packages needed
import tweepy as tp
import pandas as pd
import re
import mysql.connector
from sqlalchemy import create_engine
# !!IMPORTANT!!: You need to install the package "mysqlclient", so some functions of sqlalchemy can work.

# Personal Codes
AccessToken = ''
AccessTokenSecret = ''
APIKey = ''
APIKeySecret = ''

# Tweepy functions to authenticate our developer's account / app.
auth = tp.OAuthHandler(APIKey, APIKeySecret)
auth.set_access_token(AccessToken, AccessTokenSecret)
api = tp.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

# Connect to MySQL and create a database, using 'mysql.connector'
db = mysql.connector.connect(host="localhost", user="root", password="")
mycursor = db.cursor()
mycursor.execute("CREATE DATABASE IF NOT EXISTS sentimentdb")
# Close MySQL connection, using 'mysql.connector'
db.close()

# Connect to MySQL so we can insert later a dataframe as it is, using sqlalchemy
engine = create_engine('mysql://root:@localhost/sentimentdb?charset=utf8', encoding='utf-8')


# Now the same procedure will be followed for the 3 categories: Tesla, Toyota, and BMW.
# Hashtag comments will only be used for the first category as the procedure is the same for the other 2.

##############
#  1. Tesla  #
##############

# Create empty list to fill later, through iterations
tweets = []
i = 1
print("1. Tesla")
# Api.search looks for tweets based on the info we are feeding it.
# We include tweets that contain the: "#Tesla".
# We exclude: Retweets, Languages other than english.
# .items(n) is the number if iterations. It will extract 'n' number of tweets.
for tweet in tp.Cursor(api.search,
                       q="#Tesla",
                       lang="en",
                       tweet_mode="extended",
                       include_rts=0).items(30):

    # We iterate and print i, just to see the progress of our extraction
    print(i)
    i += 1

    # We create a list with the information asked in this order:
    # User name, Number of followers, Number of tweets, Number of tweet's retweets, Text, Date, Location, Hashtags
    try:
        data = [tweet.user.screen_name,
                tweet.user.followers_count,
                tweet.user.statuses_count,
                tweet.retweet_count,
                re.sub(r"(@|http)\S+", "", tweet.full_text),
                tweet.created_at,
                tweet.place,
                tweet.entities['hashtags']]

        # Transform list into tuple
        data = tuple(data)
        # Append it to the "tweets" list
        tweets.append(data)

    # Exclude error: "Sorry, that page does not exist"
    except tp.TweepError as e:
        print(e.reason)
        continue

    # Break iteration
    except StopIteration:
        break

# Transform list to a pandas Data Frame
tesla = pd.DataFrame(tweets,
                     columns=['User', 'Followers_U', 'Tweets_U', 'Retweets', 'Text', 'Date', 'Location', 'Hashtags'])

# Create a CSV out of the Data Frame
tesla.to_csv(path_or_buf='C:/Users/User/Desktop/1.Tesla.csv')

# Insert dataframe to MySQL table 'Tesla'.
tesla = pd.read_csv('C:/Users/User/Desktop/1.Tesla.csv', encoding='utf-8')
tesla.to_sql('tesla', con=engine, if_exists='append', index=False)


###############
#  2. Toyota  #
###############

tweets = []
i = 1
print("")
print("2. Toyota")
for tweet in tp.Cursor(api.search,
                       q="#Toyota",
                       lang="en",
                       tweet_mode="extended",
                       include_rts=0).items(30):
    print(i)
    i += 1

    try:
        data = [tweet.user.screen_name,
                tweet.user.followers_count,
                tweet.user.statuses_count,
                tweet.retweet_count,
                re.sub(r"(@|http)\S+", "", tweet.full_text),
                tweet.created_at,
                tweet.place,
                tweet.entities['hashtags']]

        data = tuple(data)
        tweets.append(data)

    except tp.TweepError as e:
        print(e.reason)
        continue

    except StopIteration:
        break

toyota = pd.DataFrame(tweets,
                      columns=['User', 'Followers_U', 'Tweets_U', 'Retweets', 'Text', 'Date', 'Location', 'Hashtags'])
toyota.to_csv(path_or_buf='C:/Users/User/Desktop/2.Toyota.csv')
toyota = pd.read_csv('C:/Users/User/Desktop/2.Toyota.csv', encoding='utf-8')
toyota.to_sql('toyota', con=engine, if_exists='append', index=False)

############
#  3. BMW  #
############

tweets = []
i = 1
print("")
print("3. BMW")
for tweet in tp.Cursor(api.search,
                       q="#BMW",
                       lang="en",
                       tweet_mode="extended",
                       include_rts=0).items(30):
    print(i)
    i += 1

    try:
        data = [tweet.user.screen_name,
                tweet.user.followers_count,
                tweet.user.statuses_count,
                tweet.retweet_count,
                re.sub(r"(@|http)\S+", "", tweet.full_text),
                tweet.created_at,
                tweet.place,
                tweet.entities['hashtags']]

        data = tuple(data)
        tweets.append(data)

    except tp.TweepError as e:
        print(e.reason)
        continue

    except StopIteration:
        break

bmw = pd.DataFrame(tweets,
                   columns=['User', 'Followers_U', 'Tweets_U', 'Retweets', 'Text', 'Date', 'Location', 'Hashtags'])
bmw.to_csv(path_or_buf='C:/Users/User/Desktop/3.BMW.csv')
bmw = pd.read_csv('C:/Users/User/Desktop/3.BMW.csv', encoding='utf-8')
bmw.to_sql('bmw', con=engine, if_exists='append', index=False)

# Close MySQL connection, using sqlalchemy
engine.dispose()


# 2. Sentiment

In [ ]:
import numpy as np
import pandas as pd
import mysql.connector
from textblob import TextBlob
from sqlalchemy import create_engine
# !!IMPORTANT!!: You need to install the package "mysqlclient", so some functions of sqlalchemy can work.


# We read the CSVs
tesla = pd.read_csv("D:/1. MASTER/5. Programming for Data Science/2. Homework/1.Tesla.csv", encoding='utf-8')
toyota = pd.read_csv("D:/1. MASTER/5. Programming for Data Science/2. Homework/2.Toyota.csv", encoding='utf-8')
bmw = pd.read_csv("D:/1. MASTER/5. Programming for Data Science/2. Homework/3.BMW.csv", encoding='utf-8')

# We add through iteration, the sentiment polarity for each text, as an extra column in our dataframes.
tesla['Polarity'] = tesla.apply(lambda x: TextBlob(x['Text']).sentiment.polarity, axis=1)
toyota['Polarity'] = toyota.apply(lambda x: TextBlob(x['Text']).sentiment.polarity, axis=1)
bmw['Polarity'] = bmw.apply(lambda x: TextBlob(x['Text']).sentiment.polarity, axis=1)

# We create a column named "Sentiment" which classifies text based on their Polarity.
#  1 = Positive
# -1 = Negative
#  0 = Neutral
tesla['Sentiment'] = np.nan
toyota['Sentiment'] = np.nan
bmw['Sentiment'] = np.nan

for i in range(0, 420):
    if tesla.iloc[i, 9] > 0:
        tesla.iloc[i, 10] = 1
    elif tesla.iloc[i, 9] < 0:
        tesla.iloc[i, 10] = -1
    else:
        tesla.iloc[i, 10] = 0

for i in range(0, 420):
    if toyota.iloc[i, 9] > 0:
        toyota.iloc[i, 10] = 1
    elif toyota.iloc[i, 9] < 0:
        toyota.iloc[i, 10] = -1
    else:
        toyota.iloc[i, 10] = 0

for i in range(0, 420):
    if bmw.iloc[i, 9] > 0:
        bmw.iloc[i, 10] = 1
    elif bmw.iloc[i, 9] < 0:
        bmw.iloc[i, 10] = -1
    else:
        bmw.iloc[i, 10] = 0

# Firstly, the first column which is unnamed, we named it 'Id' using the notebook application. Then:
# We fix the 'Id' column. It is the python's indexing. So This column has values 0-29 for every day.
# We change it to 1 to 420 for each table
for i in range(0, 420):
    tesla.iloc[i, 0] = i+1
    toyota.iloc[i, 0] = i+1
    bmw.iloc[i, 0] = i+1

# We replace the existing raw dataframes in the MySQL database, with the dataframes with the extra sentiment columns.
# Same procedure as described in "1. Scraping", but instead of 'append' we use 'replace'.
db = mysql.connector.connect(host="localhost", user="root", password="")
mycursor = db.cursor()
mycursor.execute("CREATE DATABASE IF NOT EXISTS sentimentdb")
db.close()
engine = create_engine('mysql://root:@localhost/sentimentdb?charset=utf8', encoding='utf-8')
tesla.to_sql('tesla', con=engine, if_exists='replace', index=False)
toyota.to_sql('toyota', con=engine, if_exists='replace', index=False)
bmw.to_sql('bmw', con=engine, if_exists='replace', index=False)
engine.dispose()

# We export the final CSVs
tesla.to_csv("D:/1. MASTER/5. Programming for Data Science/2. Homework/1.Tesla F.csv", index=False)
toyota.to_csv("D:/1. MASTER/5. Programming for Data Science/2. Homework/2.Toyota F.csv", index=False)
bmw.to_csv("D:/1. MASTER/5. Programming for Data Science/2. Homework/3.BMW F.csv", index=False)
